In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import json

In [ ]:
from workspace import nometools as nome
from workspace import utils

In [ ]:
main_path = Path('..')
data_path = main_path / 'Data'
steric_path = main_path / 'superposition' / 'clash_7V9J'
intersect_path = main_path / 'Data' / 'intersect_regions'
sliding_path = main_path / 'Data' / 'sliding_window_7v9j'

ref_path = data_path / 'GRCh38genome'
transcriptome_path = data_path / 'transcriptome'

#### Normalize Steric Clash

In [ ]:
infile = steric_path / 'x_y_dict'
with open(infile, 'rb') as fin:
    x_y_clash_dict = pickle.load(fin)

x_y_clash_dict_norm_ = utils.normalize_clash_dict(x_y_clash_dict)

meth_thres_range = [0, 10, 20]
clash_thres_range = [5, 10, 20, 50]

params = []
for clash_thres in clash_thres_range:
    for meth_thres in meth_thres_range:
        k = "c" + str(clash_thres) + "m" + str(meth_thres)
        params.append(k)

In [ ]:
plt.plot(x_y_clash_dict.values(), marker='o')
plt.plot(x_y_clash_dict_norm_.values(), marker='o')

In [ ]:
x = list(x_y_clash_dict.keys())
y = list(x_y_clash_dict.values())

fig = plt.figure(figsize=(18, 7))
ax = plt.subplot(111)

#plt.title('Superposition of DNMT1 (3PTA) and the nucleosome (1KX5)')
# print(sum(np.array(y) < 5))
ax.plot(x, y, linestyle="-", marker="o", linewidth=2)
# ax.plot(np.array(x)[np.array(y) < 5], np.array(y)[np.array(y) < 5], '*', color='red')
for pos in range(0,len(x)):
    #ax.text(x[pos]+0.3,y[pos],rmsd_list[pos])
    ax.text(x[pos]+0.2, y[pos]+0.7, x[pos], fontsize=12)
    
plt.xlim(0,140)
plt.xticks(range(0,len(x)+5,5), fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Nucleosome position [bp]", fontsize=18)
plt.ylabel("DNMT1 atoms clashing [%]", fontsize=18)
plt.ylim(top=100)

ax.axvline(x=74,c="grey",linewidth=2,linestyle="--")#4169e1

In [ ]:
len(x_y_clash_dict.keys()), len(x_y_clash_dict_norm_.keys())

#### Expression and Map

In [ ]:
df_expression = pd.read_csv(transcriptome_path / 'LTC_HepG2_MonoCal_FA.rep1.TPM.txt', sep='\t', names=['id', 'expression'], header=0)
df_expression['gene_id'] = df_expression['id'].str.split(':').str[0]
df_expression.head()

In [ ]:
# df_expression = pd.read_csv(transcriptome_path / 'GSM2343347.tsv', sep='\t')
# df_expression.head()

In [ ]:
with open(ref_path / 'transcript_to_gene.json', 'r') as file:
    transcript_to_gene = json.load(file)

In [ ]:
gene_to_transcript = {v.split('.')[0]:k for k,v in transcript_to_gene.items()}

In [ ]:
gene_to_transcript

In [ ]:
mask_expressed = df_expression['expression'] > 0
transcript_expressed = [gene_to_transcript[gene] for gene in df_expression.loc[mask_expressed, 'gene_id'] if gene in gene_to_transcript]
transcript_not_expressed = [gene_to_transcript[gene] for gene in df_expression.loc[~mask_expressed, 'gene_id'] if gene in gene_to_transcript]

In [ ]:
len(transcript_expressed), len(transcript_not_expressed)

#### Cohen's D and P values

In [ ]:
import scipy
from scipy import stats

def calculate_cohens_d(list_EXP,list_RAND):
    mEXP = np.mean(list_EXP)
    sdEXP = np.std(list_EXP)
    
    mRAND = np.mean(list_RAND)
    sdRAND = np.std(list_RAND)
    
    denom = np.sqrt(float(sdEXP**2 + sdRAND**2)/2.0)
    cohens_d = float(mEXP-mRAND)/float(denom + 1e-6)
    
    return cohens_d

def make_df_p_vals_cohens_d(df_scores_EXP, df_scores_RAND, params):
    column_names = ["nbr_CpGs", "parameter", "N_EXP", "N_RAND", "mean_EXP", "median_EXP", "std_EXP", "mean_RAND",
                    "median_RAND", "std_RAND", "cohens_d", "is_normal_EXP", "is_normal_RAND", "pval_ttest",
                    "t_stat", "pval_ranksums", "pval_ks_2samp"]
    info_dict = dict()
    for col in column_names:
        info_dict[col] = []

    nbr_CpGs_list = list(set(list(df_scores_EXP["nbr_CpGs"])))

    for nbr_CpGs in nbr_CpGs_list:
        df_scores_EXP_tmp = df_scores_EXP.loc[df_scores_EXP["nbr_CpGs"] == nbr_CpGs]
        df_scores_RAND_tmp = df_scores_RAND.loc[df_scores_RAND["nbr_CpGs"] == nbr_CpGs]
                    
        for p in range(len(params)):
            par_name = params[p]
            EXP_scores = list(df_scores_EXP_tmp[par_name])
            RAND_scores = list(df_scores_RAND_tmp[par_name])
            
            info_dict["nbr_CpGs"].append(nbr_CpGs)
            info_dict["parameter"].append(par_name)
            
            info_dict["N_EXP"].append(len(EXP_scores))
            info_dict["N_RAND"].append(len(RAND_scores))
            
            info_dict["mean_EXP"].append(np.mean(EXP_scores))
            info_dict["median_EXP"].append(np.median(EXP_scores))
            info_dict["std_EXP"].append(np.std(EXP_scores))
            
            info_dict["mean_RAND"].append(np.mean(RAND_scores))
            info_dict["median_RAND"].append(np.median(RAND_scores))
            info_dict["std_RAND"].append(np.std(RAND_scores))
            
            #EFFECT SIZE
            cohens_d = calculate_cohens_d(EXP_scores, RAND_scores)
            info_dict["cohens_d"].append(cohens_d)
            
            #STAT TESTS
            #is normal distributed? This function tests the null hypothesis that a sample comes from a normal distribution. If small -> ost likely not normal dustributed
            pval_normal_EXP = stats.normaltest(EXP_scores)[1] if len(EXP_scores) >= 8 else -1
            pval_normal_RAND = stats.normaltest(RAND_scores)[1] if len(EXP_scores) >= 8 else -1
            
            #students t
            ttest_res = stats.ttest_ind(EXP_scores,RAND_scores,equal_var = False)
            t_stat = ttest_res[0]
            p_val_ttest = float(ttest_res[1])/2.0
            
            #ranksums, kstest
            ranksums = scipy.stats.ranksums(EXP_scores,RAND_scores)[1]
            ks_2samp = scipy.stats.ks_2samp(EXP_scores,RAND_scores)[1]
    
            info_dict["is_normal_EXP"].append(pval_normal_EXP)
            info_dict["is_normal_RAND"].append(pval_normal_RAND)
            info_dict["pval_ttest"].append(p_val_ttest)
            info_dict["t_stat"].append(t_stat)
            info_dict["pval_ranksums"].append(ranksums)
            info_dict["pval_ks_2samp"].append(ks_2samp)
            
    #Built dataframe
    df = pd.DataFrame(0, index = np.arange(len(info_dict["nbr_CpGs"])),columns = column_names)
    for feat in column_names:
        df[feat] = info_dict[feat]

    return df

In [ ]:
region = 'promoter'
# region = 'intron.1.start'
# region = 'intron.1.end'
# region = 'intron.2.start'

In [ ]:
expression = False

In [ ]:
df_NDR_score_exp = pd.read_csv(sliding_path / f'{region}.df_NDR_score_exp.csv')
df_NDR_score_random = pd.read_csv(sliding_path / f'{region}.df_NDR_score_random.csv')
df_NOR_score_exp = pd.read_csv(sliding_path / f'{region}.df_NOR_score_exp.csv')
df_NOR_score_random = pd.read_csv(sliding_path / f'{region}.df_NOR_score_random.csv')

In [ ]:
df_NDR_score_exp.head()

In [ ]:
df_NDR_score_exp['refid'].unique().shape, df_NOR_score_exp['refid'].unique().shape

In [ ]:
if expression:
    df_NDR_score_exp = df_NDR_score_exp[df_NDR_score_exp['refid'].isin(transcript_expressed)]
    df_NDR_score_random = df_NDR_score_random[df_NDR_score_random['refid'].isin(transcript_expressed)]
    df_NOR_score_exp = df_NOR_score_exp[df_NOR_score_exp['refid'].isin(transcript_expressed)]
    df_NOR_score_random = df_NOR_score_random[df_NOR_score_random['refid'].isin(transcript_expressed)]
else:
    df_NDR_score_exp = df_NDR_score_exp[df_NDR_score_exp['refid'].isin(transcript_not_expressed)]
    df_NDR_score_random = df_NDR_score_random[df_NDR_score_random['refid'].isin(transcript_not_expressed)]
    df_NOR_score_exp = df_NOR_score_exp[df_NOR_score_exp['refid'].isin(transcript_not_expressed)]
    df_NOR_score_random = df_NOR_score_random[df_NOR_score_random['refid'].isin(transcript_not_expressed)]

In [ ]:
df_NDR_score_exp['refid'].unique().shape, df_NOR_score_exp['refid'].unique().shape

In [ ]:
df_NDR_pvalues = make_df_p_vals_cohens_d(df_NDR_score_exp, df_NDR_score_random, params)

In [ ]:
df_NOR_pvalues = make_df_p_vals_cohens_d(df_NOR_score_exp, df_NOR_score_random, params)

In [ ]:
df_NDR_pvalues.head()

In [ ]:
plt.figure(figsize=(15,7))
groups = df_NDR_pvalues.groupby(by='parameter')
for par, df_tmp in groups:
    plt.scatter(df_tmp['nbr_CpGs'], df_tmp['N_EXP'], label=par)

# plt.axhline(y=50)
plt.xlim(-2,30)
# plt.ylim((10**1,10**6))
# plt.legend()
plt.xlabel('Number of CpGs in sliding window', fontsize=22)
plt.ylabel('Sample size for Cohen’s D', fontsize=22)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.yscale('log')

In [ ]:
plt.figure(figsize=(15,7))
groups = df_NOR_pvalues.groupby(by='parameter')
for par, df_tmp in groups:
    plt.scatter(df_tmp['nbr_CpGs'], df_tmp['N_EXP'], label=par)
plt.xlim(-2,30)
# plt.legend()
plt.xlabel('Number of CpGs in sliding window', fontsize=22)
plt.ylabel('Sample size for Cohen’s D', fontsize=22)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.yscale('log')

In [ ]:
def plot_nbr_CpGs_cohensd(df_values, params):
    plt.figure(figsize=(15,7))
    ax = plt.subplot(1,1,1)
    
    nbr_CpGs_list = sorted(list(set(list(df_values["nbr_CpGs"]))))

    for par_name in params:
        
        df_values_tmp = df_values.loc[df_values["parameter"] == par_name]
        
        x_nbr_cpg_vals = list(df_values_tmp["nbr_CpGs"])
        y_cohens_d_vals = list(df_values_tmp["cohens_d"])
        
        if "m0" in par_name:
            c = "#117A65"
        if "m10" in par_name:
            c = "#45B39D"
        if "m20" in par_name:
            c = "#EB984E"
        if "c5" in par_name:
            m = "*"
            ms = 15
        if "c10" in par_name:
            m = "^"
            ms = 10
        if "c20" in par_name:
            m = "s"
            ms = 10
        if "c50" in par_name:
            m = "o"
            ms = 10
        
        
        plt.plot(x_nbr_cpg_vals, y_cohens_d_vals, linestyle="-", color=c, marker=m, markersize=ms, label=par_name)
    
    
    plt.axhline(y=0.2, linewidth=1, color = '#2C3E50',linestyle='--')
    plt.axhline(y=0.5, linewidth=1, color = '#2C3E50',linestyle='--')
    plt.axhline(y=0.8, linewidth=1, color = '#2C3E50',linestyle='--')
    
    e = 0.02
    ax.text(-1.8,0.2+e, "Small ES")
    ax.text(-1.8,0.5+e, "Medium ES")
    ax.text(-1.8,0.8+e, "Large ES")

    ax.set_ylabel("Cohen's D", fontsize=22)
    ax.set_xlabel("Number of CpGs in sliding window", fontsize=22)   

    plt.xlim(-2,30)
    plt.ylim(-5,5)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)

    legend = ax.legend(loc="lower left", ncol=4, frameon = 1, prop={'size':18})
    legend.get_frame().set_facecolor('white')

In [ ]:
plot_nbr_CpGs_cohensd(df_NDR_pvalues, params)

In [ ]:
plot_nbr_CpGs_cohensd(df_NOR_pvalues, params)

In [ ]:
def plot_pvals_cohensd(df_values,params):
    
    plt.figure(figsize=(15,7))
    ax = plt.subplot(1,1,1)
    
    nbr_CpGs_list = sorted(list(set(list(df_values["nbr_CpGs"]))))
    
    for par_name in params:
        df_values_tmp = df_values.loc[df_values["parameter"] == par_name]
        
        x_nbr_cpg_vals = list(df_values_tmp["nbr_CpGs"])
        y_pvals_vals = list(df_values_tmp["pval_ranksums"])
        
        y_pvals_log = [-np.log10(p_val) if p_val != 0.0 else 310 for p_val in y_pvals_vals]
        # print(min([p_val for p_val in y_pvals_vals]))
        # y_pvals_log = [-np.log10(p_val) for p_val in y_pvals_vals]
        
        if "m0" in par_name:
            c = "#117A65"
        if "m10" in par_name:
            c = "#45B39D"
        if "m20" in par_name:
            c = "#EB984E"
        if "c5" in par_name:
            m = "*"
            ms = 15
        if "c10" in par_name:
            m = "^"
            ms = 10
        if "c20" in par_name:
            m = "s"
            ms = 10
        if "c50" in par_name:
            m = "o"
            ms = 10
        
        
        plt.plot(x_nbr_cpg_vals,y_pvals_log,linestyle="-",color=c,marker=m,markersize=ms,label=par_name)
    
    
    #plt.axhline(y=-np.log10(0.05), linewidth=1, color = '#5D6D7E',linestyle='-')
    #plt.axhline(y=-np.log10(0.01), linewidth=1, color = '#515A5A',linestyle='-')

    ax.set_ylabel("-log10(P-value)", fontsize=22)
    ax.set_xlabel("Number of CpGs in sliding window", fontsize=22)   

    plt.xlim(-2, 30)
    plt.ylim(0, 450)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)

    legend = ax.legend(loc="upper right",ncol=4,frameon = 1,prop={'size':18})
    legend.get_frame().set_facecolor('white')

In [ ]:
plot_pvals_cohensd(df_NDR_pvalues, params)

In [ ]:
plot_pvals_cohensd(df_NOR_pvalues, params)

#### Cohen's D Calculation Table

In [ ]:
import scipy
from scipy import stats

def calculate_cohens_d(list_EXP,list_RAND):
    mEXP = np.mean(list_EXP)
    sdEXP = np.std(list_EXP)
    
    mRAND = np.mean(list_RAND)
    sdRAND = np.std(list_RAND)
    
    denom = np.sqrt(float(sdEXP**2 + sdRAND**2)/2.0)
    cohens_d = float(mEXP-mRAND)/float(denom + 1e-6)
    
    return cohens_d

def make_df_cohens_d(df_scores_EXP, df_scores_RAND, params):
    column_names = ["parameter", "N_EXP", "N_RAND", "mean_EXP", "median_EXP", "std_EXP", "mean_RAND",
                    "median_RAND", "std_RAND", "cohens_d", "is_normal_EXP", "is_normal_RAND", "pval_ttest",
                    "t_stat", "pval_ranksums", "pval_ks_2samp"]
    info_dict = dict()
    for col in column_names:
        info_dict[col] = []
    
    df_scores_EXP_tmp = df_scores_EXP.loc[df_scores_EXP['nbr_CpGs'].between(10,20)]
    df_scores_RAND_tmp = df_scores_RAND.loc[df_scores_RAND['nbr_CpGs'].between(10,20)]
                    
    for p in range(len(params)):
        par_name = params[p]
        EXP_scores = list(df_scores_EXP_tmp[par_name])
        RAND_scores = list(df_scores_RAND_tmp[par_name])
        
        # info_dict["nbr_CpGs"].append(nbr_CpGs)
        info_dict["parameter"].append(par_name)
        
        info_dict["N_EXP"].append(len(EXP_scores))
        info_dict["N_RAND"].append(len(RAND_scores))
        
        info_dict["mean_EXP"].append(np.mean(EXP_scores))
        info_dict["median_EXP"].append(np.median(EXP_scores))
        info_dict["std_EXP"].append(np.std(EXP_scores))
        
        info_dict["mean_RAND"].append(np.mean(RAND_scores))
        info_dict["median_RAND"].append(np.median(RAND_scores))
        info_dict["std_RAND"].append(np.std(RAND_scores))
        
        #EFFECT SIZE
        cohens_d = calculate_cohens_d(EXP_scores, RAND_scores)
        info_dict["cohens_d"].append(cohens_d)
        
        #STAT TESTS
        #is normal distributed? This function tests the null hypothesis that a sample comes from a normal distribution. If small -> ost likely not normal dustributed
        pval_normal_EXP = stats.normaltest(EXP_scores)[1] if len(EXP_scores) >= 8 else -1
        pval_normal_RAND = stats.normaltest(RAND_scores)[1] if len(EXP_scores) >= 8 else -1
        
        #students t
        ttest_res = stats.ttest_ind(EXP_scores,RAND_scores,equal_var = False)
        t_stat = ttest_res[0]
        p_val_ttest = float(ttest_res[1])/2.0
        
        #ranksums, kstest
        ranksums = scipy.stats.ranksums(EXP_scores,RAND_scores)[1]
        ks_2samp = scipy.stats.ks_2samp(EXP_scores,RAND_scores)[1]

        info_dict["is_normal_EXP"].append(pval_normal_EXP)
        info_dict["is_normal_RAND"].append(pval_normal_RAND)
        info_dict["pval_ttest"].append(p_val_ttest)
        info_dict["t_stat"].append(t_stat)
        info_dict["pval_ranksums"].append(ranksums)
        info_dict["pval_ks_2samp"].append(ks_2samp)
            
    #Built dataframe
    df = pd.DataFrame(0, index = np.arange(len(info_dict["parameter"])),columns = column_names)
    for feat in column_names:
        df[feat] = info_dict[feat]

    return df

In [ ]:
# region = 'promoter'
# region = 'intron.1.start'
# region = 'intron.1.end'
# region = 'intron.2.start'

In [ ]:
# df_NDR_score_exp = pd.read_csv(sliding_path / f'{region}.df_NDR_score_exp.csv')
# df_NDR_score_random = pd.read_csv(sliding_path / f'{region}.df_NDR_score_random.csv')
# df_NOR_score_exp = pd.read_csv(sliding_path / f'{region}.df_NOR_score_exp.csv')
# df_NOR_score_random = pd.read_csv(sliding_path / f'{region}.df_NOR_score_random.csv')

In [ ]:
# df_NDR_pvalues = make_df_cohens_d(df_NDR_score_exp, df_NDR_score_random, params)

In [ ]:
# df_NOR_pvalues = make_df_cohens_d(df_NOR_score_exp, df_NOR_score_random, params)

In [ ]:
# df_NDR_pvalues

In [ ]:
# df_NOR_pvalues

In [ ]:
expression = False
regions = ['promoter', 'intron.1.start', 'intron.1.end', 'intron.2.start']
labels = ['promoter', 'start of 1st intron', 'end of 1st intron', 'start of 2nd intron']
table = {}
for label, region in zip(labels, regions):
    print(region, expression)
    df_NDR_score_exp = pd.read_csv(sliding_path / f'{region}.df_NDR_score_exp.csv')
    df_NDR_score_random = pd.read_csv(sliding_path / f'{region}.df_NDR_score_random.csv')
    df_NOR_score_exp = pd.read_csv(sliding_path / f'{region}.df_NOR_score_exp.csv')
    df_NOR_score_random = pd.read_csv(sliding_path / f'{region}.df_NOR_score_random.csv')

    if expression:
        print(f"NDR Exp unique transcripts: {df_NDR_score_exp['refid'].unique().shape}")
        df_NDR_score_exp = df_NDR_score_exp[df_NDR_score_exp['refid'].isin(transcript_expressed)]
        print(f"NDR Exp Expressed unique transcripts: {df_NDR_score_exp['refid'].unique().shape}")

        print(f"NDR Rand unique transcripts: {df_NDR_score_random['refid'].unique().shape}")
        df_NDR_score_random = df_NDR_score_random[df_NDR_score_random['refid'].isin(transcript_expressed)]
        print(f"NDR Rand Expressed unique transcripts: {df_NDR_score_random['refid'].unique().shape}")

        print(f"NOR Exp unique transcripts: {df_NOR_score_exp['refid'].unique().shape}")
        df_NOR_score_exp = df_NOR_score_exp[df_NOR_score_exp['refid'].isin(transcript_expressed)]
        print(f"NOR Exp Expressed unique transcripts: {df_NOR_score_exp['refid'].unique().shape}")

        print(f"NOR Rand unique transcripts: {df_NOR_score_random['refid'].unique().shape}")
        df_NOR_score_random = df_NOR_score_random[df_NOR_score_random['refid'].isin(transcript_expressed)]
        print(f"NOR Rand Expressed unique transcripts: {df_NOR_score_random['refid'].unique().shape}")
    else:
        print(f"NDR Exp unique transcripts: {df_NDR_score_exp['refid'].unique().shape}")
        df_NDR_score_exp = df_NDR_score_exp[df_NDR_score_exp['refid'].isin(transcript_not_expressed)]
        print(f"NDR Exp Not Expressed unique transcripts: {df_NDR_score_exp['refid'].unique().shape}")

        print(f"NDR Rand unique transcripts: {df_NDR_score_random['refid'].unique().shape}")
        df_NDR_score_random = df_NDR_score_random[df_NDR_score_random['refid'].isin(transcript_not_expressed)]
        print(f"NDR Rand Not Expressed unique transcripts: {df_NDR_score_random['refid'].unique().shape}")

        print(f"NOR Exp unique transcripts: {df_NOR_score_exp['refid'].unique().shape}")
        df_NOR_score_exp = df_NOR_score_exp[df_NOR_score_exp['refid'].isin(transcript_not_expressed)]
        print(f"NOR Exp Not Expressed unique transcripts: {df_NOR_score_exp['refid'].unique().shape}")

        print(f"NOR Rand unique transcripts: {df_NOR_score_random['refid'].unique().shape}")
        df_NOR_score_random = df_NOR_score_random[df_NOR_score_random['refid'].isin(transcript_not_expressed)]
        print(f"NOR Rand Not Expressed unique transcripts: {df_NOR_score_random['refid'].unique().shape}")

    df_NDR_pvalues = make_df_cohens_d(df_NDR_score_exp, df_NDR_score_random, params)
    df_NOR_pvalues = make_df_cohens_d(df_NOR_score_exp, df_NOR_score_random, params)

    assert df_NDR_pvalues.shape[0] == df_NOR_pvalues.shape[0]
    assert df_NDR_pvalues['parameter'].to_list() == df_NOR_pvalues['parameter'].to_list()

    table[label] = {
        'Parameter': df_NDR_pvalues['parameter'].to_list(),
        'HNDRs': df_NOR_pvalues['cohens_d'].to_list(),
        'LNDRs': df_NDR_pvalues['cohens_d'].to_list()
    }

In [ ]:
df_table = pd.DataFrame(table).T.explode(['Parameter', 'LNDRs', 'HNDRs'])

In [ ]:
df_table

In [ ]:
print(df_table.to_latex(float_format="{:.2f}".format))

In [ ]:
df_table[df_table['Parameter'].isin(['c5m0', 'c5m10', 'c5m20'])]

In [ ]:
print(df_table[df_table['Parameter'].isin(['c5m0', 'c5m10', 'c5m20'])].to_latex(float_format="{:.2f}".format))

#### Histograms

In [ ]:
region = 'promoter'
# region = 'intron.1.start'
# region = 'intron.1.end'
# region = 'intron.2.start'

In [ ]:
df_NDR_score_exp = pd.read_csv(sliding_path / f'{region}.df_NDR_score_exp.csv')
df_NDR_score_random = pd.read_csv(sliding_path / f'{region}.df_NDR_score_random.csv')
df_NOR_score_exp = pd.read_csv(sliding_path / f'{region}.df_NOR_score_exp.csv')
df_NOR_score_random = pd.read_csv(sliding_path / f'{region}.df_NOR_score_random.csv')

In [ ]:
df_NDR_score_exp.columns

In [ ]:
fig, axs = plt.subplots(3, 4, figsize=(20,12), layout='constrained')
order = ['c5m0', 'c10m0', 'c20m0', 'c50m0', 'c5m10', 'c10m10', 'c20m10', 'c50m10', 'c5m20', 'c10m20', 'c20m20', 'c50m20']

for ax, thresh in zip(axs.flatten(), order):
    ax.hist(df_NDR_score_random.loc[df_NDR_score_random['nbr_CpGs'].between(10,20), thresh], density=True, alpha=0.5, bins=np.linspace(0,1,20), label='random')
    ax.hist(df_NDR_score_exp.loc[df_NDR_score_exp['nbr_CpGs'].between(10,20), thresh], density=True, alpha=0.3, bins=np.linspace(0,1,20), label='experimental')
    ax.set_title(thresh, fontsize=20)
    ax.set_xlabel('match-score', fontsize=16)
    ax.set_ylabel('density', fontsize=16)
    ax.legend(fontsize=12)

In [ ]:
fig, axs = plt.subplots(3, 4, figsize=(20,12), layout='constrained')
order = ['c5m0', 'c10m0', 'c20m0', 'c50m0', 'c5m10', 'c10m10', 'c20m10', 'c50m10', 'c5m20', 'c10m20', 'c20m20', 'c50m20']

for ax, thresh in zip(axs.flatten(), order):
    ax.hist(df_NOR_score_random.loc[df_NOR_score_random['nbr_CpGs'].between(10,20), thresh], density=True, alpha=0.5, bins=np.linspace(0,1,20), label='random')
    ax.hist(df_NOR_score_exp.loc[df_NOR_score_exp['nbr_CpGs'].between(10,20), thresh], density=True, alpha=0.3, bins=np.linspace(0,1,20), label='experimental')
    ax.set_title(thresh, fontsize=20)
    ax.set_xlabel('match-score', fontsize=16)
    ax.set_ylabel('density', fontsize=16)
    ax.legend(fontsize=12)